In [ ]:
# === 1. Install & Imports ===
!pip install --quiet mne pyxdf

import os
import numpy as np
import pandas as pd
import mne
import pyxdf
from mne.time_frequency import psd_array_welch

# === 2. Helper Functions ===

def load_eeg_from_xdf(path):
    streams, _ = pyxdf.load_xdf(path)
    # Pick the stream whose type is exactly 'EEG'
    eegs = [s for s in streams if s['info']['type'][0] == 'EEG']
    if not eegs:
        raise ValueError(f"No EEG stream in {os.path.basename(path)}")
    e = eegs[0]
    data = np.array(e['time_series']).T
    sfreq = float(e['info']['nominal_srate'][0])
    chs = [ch['label'][0] for ch in e['info']['desc'][0]['channels'][0]['channel']]
    info = mne.create_info(ch_names=chs, sfreq=sfreq, ch_types=['eeg']*len(chs))
    return mne.io.RawArray(data, info)

def preprocess(raw):
    sfreq = raw.info['sfreq']
    # Check Nyquist: need sfreq/2 > 40 to do a 1–40 Hz bandpass
    if sfreq / 2 <= 40:
        print(f"⚠ Sampling rate {sfreq:.1f} Hz too low for 40 Hz filter; skipping this file.")
        return None
    raw.filter(1., 40., fir_design='firwin')
    raw.set_eeg_reference('average', projection=True)
    raw.interpolate_bads()
    return raw

def compute_band_power_from_data(data, sfreq):
    # Drop NaN or flat channels
    mask = (~np.isnan(data).any(axis=1)) & (data.std(axis=1) > 1e-12)
    data = data[mask]
    if data.size == 0:
        return dict.fromkeys(['delta','theta','alpha','beta','gamma'], np.nan)
    # Remove DC offset
    data = data - data.mean(axis=1, keepdims=True)
    n_fft = min(2048, data.shape[1])
    psds, freqs = psd_array_welch(data, sfreq=sfreq, fmin=1, fmax=40, n_fft=n_fft)
    return {
        'delta': psds[:, (freqs>=1)  & (freqs<4)].mean(),
        'theta': psds[:, (freqs>=4)  & (freqs<8)].mean(),
        'alpha': psds[:, (freqs>=8)  & (freqs<13)].mean(),
        'beta' : psds[:, (freqs>=13) & (freqs<30)].mean(),
        'gamma': psds[:, (freqs>=30)].mean()
    }

# === 3. Process participant P02 ===

participant = 'P04'
base_path   = f"/content/nanoe_eeg_data/{participant}"
results     = []

# 3a) Eye‐closure & Eye‐opening (session‐level)
for task in ['eye_closure','eye_opening']:
    for cond in ['with','without']:
        fpath = os.path.join(base_path, f"{task}_{cond}.xdf")
        if not os.path.isfile(fpath):
            print("Missing:", fpath)
            continue
        raw = load_eeg_from_xdf(fpath)
        raw = preprocess(raw)
        if raw is None:
            continue
        bp = compute_band_power_from_data(raw.get_data(), raw.info['sfreq'])
        bp.update({
            'participant': participant,
            'task': task,
            'condition': cond,
            'event': None
        })
        results.append(bp)

# 3b) Driving (event‐wise on full recording)
for cond in ['with','without']:
    fpath = os.path.join(base_path, f"driving_{cond}.xdf")
    if not os.path.isfile(fpath):
        continue

    # 1) Load & preprocess full raw
    raw_full = load_eeg_from_xdf(fpath)
    raw_full = preprocess(raw_full)
    if raw_full is None:
        continue
    sfreq   = raw_full.info['sfreq']
    streams, _ = pyxdf.load_xdf(fpath)

    bp_full = compute_band_power_from_data(raw_full.get_data(picks='eeg'), sfreq)
    bp_full.update({
        'participant': participant,
        'task':       'driving',
        'condition':  cond,
        'event':      'entire recording'
    })
    results.append(bp_full)

    # 2) Align all marker streams to EEG
    eeg_stream = next(s for s in streams if s['info']['type'][0]=='EEG')
    eeg_ts     = np.array(eeg_stream['time_stamps'])
    eeg_start  = eeg_ts[0]

    marker_streams = [s for s in streams if s['info']['type'][0]=='Markers']
    all_ts  = np.concatenate([s['time_stamps']    for s in marker_streams])
    all_lbl = np.concatenate([[v[0] for v in s['time_series']] for s in marker_streams])
    rel_t   = all_ts - eeg_start
    samples = np.round(rel_t * sfreq).astype(int)

    # 3) For each event label, collect its sample indices (sorted)
    label_to_samples = {}
    for samp, lbl in zip(samples, all_lbl):
        if 0 <= samp < raw_full.n_times:
            label_to_samples.setdefault(lbl, []).append(samp)

    # 4) For each label, form successive intervals & compute segment-power
    for lbl, s_list in label_to_samples.items():
        s_list = sorted(s_list)
        segment_powers = []
        for start, end in zip(s_list, s_list[1:]):
            # skip if too short
            if end - start < int(sfreq*0.5):  # e.g. require at least 0.5 s
                continue
            seg = raw_full.get_data(picks='eeg')[:, start:end]
            bp_seg = compute_band_power_from_data(seg, sfreq)
            segment_powers.append(bp_seg)

        # 5) Average across all segments for this label
        if not segment_powers:
            avg_bp = dict.fromkeys(['delta','theta','alpha','beta','gamma'], np.nan)
        else:
            avg_bp = {}
            for band in ['delta','theta','alpha','beta','gamma']:
                avg_bp[band] = np.nanmean([seg[band] for seg in segment_powers])

        # 6) Add one row for this label
        avg_bp.update({
            'participant': participant,
            'task':       'driving',
            'condition':  cond,
            'event':      lbl
        })
        results.append(avg_bp)

# === 4. Aggregate & Save ===

df = pd.DataFrame(results)[
    ['delta','theta','alpha','beta','gamma','participant','task','condition','event']
]
print("\nFinal event‐wise band‐power table for", participant)
print(df)
df.to_csv(f"{participant}_eventwise_band_power.csv", index=False)
print("\nSaved to", f"{participant}_eventwise_band_power.csv")
